In [11]:
import pandas as pd
from collections import defaultdict
import json

In [22]:
#full = pd.read_csv("../data_prep/clean/cleaned.csv", skiprows=range( int(2974335 * .001), 2974335))
full = pd.read_csv("../data_prep/clean/cleaned.csv")

In [3]:
# Question: Which state has the worst drivers?
# Quantized by crashed per capita
# Data Source: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html#par_textimage_1873399417
# Population, Population Change, and Estimated Components of Population Change: April 1, 2010 to July 1, 2019 (NST-EST2019-alldata)
pop = pd.read_csv("../data_prep/other_sources/population_by_state.csv")


state_counts = defaultdict(int)
for k,v in full["State"].value_counts().to_dict().items():
    state_counts[k] = v

state_code = pop["STATECODE"]
# POPESTIMATE MEAN
cols = [f"POPESTIMATE{x}" for x in [2016, 2017, 2018, 2019]] 
pop["4yrpopsum"] = pop[cols].sum(axis=1)
# Over 4 years how many people lived in each state
state_info = pop["4yrpopsum"]
# Contains a list of all the state abbreviations
state_code
# State counts maps state code to number of accidents
state_counts
# Maps all abbreviations to the number of accidents
counts = state_code.apply(lambda code: state_counts.get(code)).fillna(1)
state_info

# Contains chart of accidents per capita
worst_drivers = pd.DataFrame(counts / state_info, columns=["AccidentPC"])
worst_drivers["State"] = state_code

worst_drivers = worst_drivers[worst_drivers.State != "NAN"].sort_values("AccidentPC", ascending=False)

In [7]:
worst_drivers
worst_drivers.to_csv("./worst_drivers_state_per_capita.csv")

In [47]:
# Per state number of crashes
#full["Roundabout"].groupby(full.State).astype(int).sum()
#full.groupby("State").value_counts()

AttributeError: Cannot access callable attribute 'astype' of 'SeriesGroupBy' objects, try using the 'apply' method

In [26]:
def make_point(lat, long, severity, time):
    time_split = time.split(' ')
    point =  {
          "geometry": {
            "type": "Point",
            "coordinates": [
              long,
              lat
            ]
          },
          "type": "Feature",
          "id": "2",
          "properties": {
            "severity": severity,
            "date": time_split[0],
            "h": int(time_split[1][:2]),
          }
        }
    return point


In [27]:
# def generate_state_data():
#     """
#     INPUT: A fully populated table with all accident data
#     OUTPUT: Fills "state_crash_viz" with 50 json files containing all the points for each state
#     """
all_states = list(full.State.unique())
all = pd.DataFrame(full[["Start_Lat", "Start_Lng", "Severity", "Start_Time"]]\
    .apply(lambda row: make_point(row[0], row[1], row[2], row[3], row[4]), axis=1))
all["State"] = full.State

for state in all["State"].unique():
    print(state)
    st_data = all[all.State == state]
    st_dict = {
        "type": "FeatureCollection",
    }
    feat = []
    for p in list(st_data[0]):
        feat.append(p)
    st_dict["features"] = feat
    
    with open(f'./state_crash_viz/{state}.json', 'w') as fp:
        json.dump(st_dict, fp)
        
    




 

    


IndexError: ('index out of bounds', 'occurred at index 0')